In [5]:
import pandas as pd
from utils.DataBaseBuffer import DataBaseBuffer
import numpy as np

In [6]:
def transform_data(data_buffer: DataBaseBuffer, data: pd.DataFrame,tags:np.array, append:bool):
    pivot_df = data.pivot(index='time', columns='asset', values='value')  
    assets = pivot_df.columns.values
    elementos_excluidos = np.setdiff1d(tags, assets)
    nuevas_columnas = {col: [np.nan] * len(pivot_df) for col in elementos_excluidos}
    pivot_df = pivot_df.assign(**nuevas_columnas)

    pivot_df = pivot_df[tags]
        
    if not append:
        pivot_df.to_csv(f"data/datat_p{data_buffer.process}_e{data_buffer.exec_id}.csv", header = True)
    else:
        pivot_df.to_csv(f"data/datat_p{data_buffer.process}_e{data_buffer.exec_id}.csv", mode='a', header = False)
    
    return pivot_df.isna().sum().sum()

In [7]:
def extract_data_of_process(process):
    data_buffer = DataBaseBuffer(process, batch_size= 1000)
    while True:
        tags = data_buffer.tags_execution()
        total_nan = 0
        data_buffer.restart()
        data_temp = pd.DataFrame([], columns =["id","time","value","asset","execution"])
        while data_buffer.next():
            data = data_buffer.current_data
            data = pd.concat([data_temp, data], ignore_index=True)
            last_time = data['time'].iloc[-1]
            data_temp = data[data["time"]==last_time]
            data = data.drop(data_temp.index)

            nans_exec = transform_data(data_buffer, data, tags, data_buffer.current_batch != 0)
            total_nan = total_nan + nans_exec

        nans_exec = transform_data(data_buffer, data_temp, tags, True)
        total_nan = total_nan + nans_exec
        print(f"total datos nans {total_nan} for exec {data_buffer.exec_id}")
        
        if not data_buffer.next_execution():
            break

In [8]:
extract_data_of_process(6)

total datos nans 20 for exec 20
